<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Обработка-текста" data-toc-modified-id="Обработка-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обработка текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузка данных

In [1]:
import pandas as pd
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [3]:
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [4]:
data.duplicated().sum()

0

In [5]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


In [6]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Данные загружены. Проведем обработку текстовых данных.

### Обработка текста

Необходимо очистить текст от нежелательных символов и провести токенизацию и лемматизацию.

In [7]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text).split()
    return ' '.join(text)

In [8]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(i) for i in nltk.word_tokenize(text.lower())]
    return ' '.join(text)

In [9]:
%%time
data['text'] = data['text'].apply(clear_text)
data['text'] = data['text'].apply(lemmatize)
data.head()

CPU times: user 1min 24s, sys: 413 ms, total: 1min 24s
Wall time: 1min 24s


,Unnamed: 0,text,toxic
0,0,explanation why the edits made under my userna...,0
1,1,d aww he match this background colour i m seem...,0
2,2,hey man i m really not trying to edit war it s...,0
3,3,more i can t make any real suggestion on impro...,0
4,4,you sir are my hero any chance you remember wh...,0


## Обучение

Разделим наши данные на две выборки: тренировочную и тестовую. При создании признаков необходимо убрать стоп-слова. 


Для обучения будут рассмотрены три модели: LogisticRegression, RandomForestClassifier, CatBoostClassifier. Для каждой обученной модели будет рассчитано значение метрики F1. Модель с наилучшим значением метрики качества F1 не меньше 0.75 будет выбрана для тестирования.

In [10]:
stopwords = set(stopwords.words('english'))

In [11]:
features_train, features_test, target_train, target_test = train_test_split(
    data['text'], data['toxic'], random_state=12345, test_size=0.25)

In [12]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit(features_train)
features_train = tf_idf.transform(features_train)
features_test = tf_idf.transform(features_test)

### LogisticRegression

In [13]:
%%time
parameters = {'C': range(1, 10), 'max_iter': range(1, 10, 2)}
model = LogisticRegression(random_state=12345, solver='liblinear')
grid_lr = GridSearchCV(model, parameters, cv=3, scoring='f1')
grid_lr.fit(features_train, target_train)
print(grid_lr.best_params_)
print(grid_lr.best_score_)

{'C': 9, 'max_iter': 9}
0.7546708240508062
CPU times: user 2min 12s, sys: 3min 35s, total: 5min 47s
Wall time: 5min 48s


### RandomForestClassifier

In [14]:
%%time
parameters = {'n_estimators': range(5, 10, 2), 'max_depth': range(200, 211, 2)}
model = RandomForestClassifier(random_state=12345)
grid_rfc = GridSearchCV(model, parameters, cv=3, scoring='f1')
grid_rfc.fit(features_train, target_train)
print(grid_rfc.best_params_)
print(grid_rfc.best_score_)

{'max_depth': 208, 'n_estimators': 9}
0.5086097160452248
CPU times: user 20min 2s, sys: 0 ns, total: 20min 2s
Wall time: 20min 3s


### CatBoostClassifier

In [ ]:
%%time
parameters = {'iterations': [200], 'verbose': [False]}
model = CatBoostClassifier(random_state=12345)
grid_rfc = GridSearchCV(model, parameters, cv=3, scoring='f1')
grid_rfc.fit(features_train, target_train)
print(grid_rfc.best_params_)
print(grid_rfc.best_score_)

Для обучения будут рассмотрены три модели: LogisticRegression, RandomForestClassifier, CatBoostClassifier.
Результаты:
- LogisticRegression: 
    * f_1 score = 0.7546708240508062
    * parametrs = {'C': 9, 'max_iter': 9}
    * time = 5min 48s
- RandomForestClassifier: 
    * f_1 score = 0.5086097160452248
    * parametrs = {'max_depth': 208, 'n_estimators': 9}
    * time = 20min 3s
- CatBoostClassifier: 
    * f_1 score = 0.7320620491209784
    * parametrs = {'iterations': 200}
    * time = 25min 50s

Для тестирования будет использована LogisticRegression, так как показатель метрики f1 удовлетворяет условию задачи.

## Тестирование

In [16]:
%%time
parameters = {'C': [9], 'max_iter': [9]}
model = LogisticRegression(random_state=12345, solver='liblinear', C=9, max_iter=9)
model.fit(features_train, target_train)
predicted = model.predict(features_test)
f1_score(target_test, predicted)

CPU times: user 2.42 s, sys: 3.46 s, total: 5.88 s
Wall time: 5.89 s


0.7859804056851111

LogisticRegression: 
* f_1 score = 0.7859804056851111
* parametrs = {'C': 9, 'max_iter': 9}
* time = 5.89 s

## Выводы

В ходе поставленной задачи необходимый результат обучения был достигнут на модели LogisticRegression.